In [52]:
from scipy.io import loadmat
import numpy as np
from tqdm import tqdm
from pandas import DataFrame
from sklearn.decomposition import PCA

# 1. Implement Decision Tree method (50 points)

In [41]:
import numpy as np


class DecisionTree:
    class Node:
        def __init__(self, feature=None, threshold=None, left=None, right=None, label=None):
            self.feature = feature
            self.threshold = threshold
            self.left = left
            self.right = right
            self.label = label
    
    def __init__(self, max_depth=5, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
    
    def fit(self, X, y):
        self.num_classes = len(np.unique(y))
        self.tree = self._grow_tree(X, y, depth=0)
    
    def predict(self, X):
        return [self._traverse_tree(x, self.tree) for x in X]
    
    def _grow_tree(self, X, y, depth):
        num_samples, num_features = X.shape
        num_labels = len(np.unique(y))
        if num_samples < self.min_samples_split or num_labels == 1 or depth >= self.max_depth:
            return DecisionTree.Node(label=self._most_common_label(y))
        best_feature, best_threshold, left_indices, right_indices = self._best_split(X, y)
        left_tree = self._grow_tree(X[left_indices], y[left_indices], depth + 1)
        right_tree = self._grow_tree(X[right_indices], y[right_indices], depth + 1)
        return DecisionTree.Node(best_feature, best_threshold, left_tree, right_tree)
    
    def _best_split(self, X, y):
        num_samples, num_features = X.shape
        best_feature, best_threshold = None, None
        best_left_indices, best_right_indices = None, None
        best_gain = 0
        parent_impurity = self._gini_impurity(y)
        for feature in tqdm(range(num_features)):
            thresholds = np.unique(X[:, feature])
            if len(thresholds) > 50:
                step_size = len(thresholds) // 50
                thresholds = thresholds[::step_size]
            for threshold in thresholds:
                left_indices = X[:, feature] < threshold
                right_indices = X[:, feature] >= threshold
                if sum(left_indices) == 0 or sum(right_indices) == 0:
                    continue
                left_impurity = self._gini_impurity(y[left_indices])
                right_impurity = self._gini_impurity(y[right_indices])
                gain = parent_impurity - (sum(left_indices) / num_samples) * left_impurity - (sum(right_indices) / num_samples) * right_impurity
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold
                    best_left_indices = left_indices
                    best_right_indices = right_indices
        return best_feature, best_threshold, best_left_indices, best_right_indices
    
    def _most_common_label(self, y):
        labels, counts = np.unique(y, return_counts=True)
        return labels[np.argmax(counts)]
    
    def _gini_impurity(self, y):
        _, counts = np.unique(y, return_counts=True)
        impurity = 1.0 - np.sum(np.square(counts / len(y)))
        return impurity
    
    def _traverse_tree(self, x, node):
        if node.label is not None:
            return node.label
        if x[node.feature] < node.threshold:
            return self._traverse_tree(x, node.left)
        else:
            return self._traverse_tree(x, node.right)


# 2. Load train and test mat files, perform Decision Tree and report acuracy on the test dataset (20 points)

In [42]:
X_train, y_train = loadmat("train.mat")['features'], loadmat("train.mat")['labels']
X_test, y_test = loadmat("test.mat")['features'], loadmat("test.mat")['labels']

traindf = DataFrame(X_train)
traindf['y'] = y_train

testdf = DataFrame(X_test)
testdf['y'] = y_test

traindf.head()

,0,1,2,3,4,5,6,7,8,9,...,991,992,993,994,995,996,997,998,999,y
0,-0.190871,-0.816247,-0.163249,-1.319873,-1.264283,-0.898107,-1.718239,-0.402687,-0.236150,0.668848,...,-0.901747,-1.341656,0.117699,-0.382750,-0.299048,-0.086813,-0.173263,-0.336787,1.453196,1
1,-1.068913,-0.663617,-0.547694,-0.623559,-0.560007,0.102838,-0.240569,-0.287237,0.665804,0.418864,...,-0.048969,-0.097370,1.037708,0.346370,-0.518450,0.494111,0.236918,-0.791503,1.660773,1
2,0.210046,-0.545538,-0.235261,0.173654,-0.195184,0.151039,-0.630156,-0.696319,-0.230632,-0.011000,...,0.054887,-0.605710,0.216495,0.231946,0.638852,0.240886,0.221060,0.326909,-0.578919,1
3,-0.334718,-0.527350,0.085251,0.387529,0.625166,-0.183794,-0.293835,-0.250859,0.344714,0.279479,...,0.189216,0.005172,0.216785,0.363003,0.116603,-0.016351,0.244184,0.479256,-0.799638,1
4,0.447034,-0.245320,0.164236,0.208489,-0.039592,0.213112,-0.285458,-0.546040,-0.249142,-0.168018,...,-0.191640,-0.378032,-0.040638,-0.157071,0.221349,0.216782,0.236749,-0.024242,-1.060235,1


In [74]:
max_depth, min_samples_split = 10, 4
dt = DecisionTree(max_depth = max_depth, min_samples_split=min_samples_split)
dt.fit(X_train, y_train)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 93.10it/s]


In [75]:
preds = dt.predict(X_test)
DT_full_acc = (y_test.flatten() == preds).sum()/len(preds)
print("Full datast accuracy :", DT_full_acc)

Full datast accuracy : 0.8966597077244259


# 3. Reduce the dimensionality of features using PCA to low dimensions (e.g., 10, 3 ) and report the accuracy of the test datasets (20 points)

### With 10 components

In [60]:
pca = PCA(n_components=10)
X_train_reduced = pca.fit_transform(X_train)
X_test_reduced = pca.transform(X_test)

max_depth, min_samples_split = 10, 4
dt = DecisionTree(max_depth = max_depth, min_samples_split=min_samples_split)
dt.fit(X_train_reduced, y_train)

preds = dt.predict(X_test_reduced)
DT10_acc = (y_test.flatten() == preds).sum()/len(preds)
print("10 components accuracy :", DT10_acc)

100%|████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 1253.45it/s]

10 components accuracy : 0.9112734864300627


### With 3 components

In [61]:
pca = PCA(n_components=3)
X_train_reduced = pca.fit_transform(X_train)
X_test_reduced = pca.transform(X_test)

max_depth, min_samples_split = 10, 4
dt = DecisionTree(max_depth = max_depth, min_samples_split=min_samples_split)
dt.fit(X_train_reduced, y_train)

preds = dt.predict(X_test_reduced)
DT3_acc = (y_test.flatten() == preds).sum()/len(preds)
print("3 Compenents accuracy :", DT3_acc)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<?, ?it/s]


100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 374.95it/s]

3 Compenents accuracy : 0.7860125260960334


### With 5 components

In [62]:
pca = PCA(n_components=5)
X_train_reduced = pca.fit_transform(X_train)
X_test_reduced = pca.transform(X_test)

max_depth, min_samples_split = 10, 4
dt = DecisionTree(max_depth = max_depth, min_samples_split=min_samples_split)
dt.fit(X_train_reduced, y_train)

preds = dt.predict(X_test_reduced)

DT5_acc = (y_test.flatten() == preds).sum()/len(preds)
print("5 compenents accuracy :", DT5_acc)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<?, ?it/s]

5 compenents accuracy : 0.9133611691022965


# 4. Compare results with your KNN model (10 points)

In [64]:
class KNN:
    def __init__(self, k):
        self.k = k
    
    def fit(self, X, y):
        self.data = X
        self.lbls = y
            
    def predict(self, X_test, dist_metric):
        preds = []
        if dist_metric == "manhattan":
            for testpoint in tqdm(X_test):
                distances = [ np.sum(abs(x-testpoint)) for x in self.data]
                preds.append(np.max(self.lbls[np.argsort(distances)][:self.k]))
        elif dist_metric == "cosine":
            for testpoint in tqdm(X_test):
                distances = [ 1-(x*testpoint)/(LA.norm(x)*LA.norm(x)) for x in self.data]
                preds.append(np.max(self.lbls[np.argsort(distances)][:self.k]))
        elif dist_metric == 'minkowski':
            for testpoint in tqdm(X_test):
                distances = [ np.sum((testpoint - x)**2) for x in self.data]
                preds.append(np.max(self.lbls[np.argsort(distances)][:self.k]))
        elif dist_metric == "euclidean":
            for testpoint in tqdm(X_test):
                distances = [ np.sum(np.sqrt((testpoint - x)**2)) for x in self.data]
                preds.append(np.max(self.lbls[np.argsort(distances)][:self.k]))
        elif dist_metric == "chebyshev":
            for testpoint in tqdm(X_test):
                distances = [ np.max(x-testpoint) for x in self.data]
                preds.append(np.max(self.lbls[np.argsort(distances)][:self.k]))
            

        return  preds

In [66]:
knn = KNN(4)
knn.fit(X_train, y_train)

knn.predict(X_test, dist_metric='minkowski')
knn_full_acc = (y_test.flatten() == preds).sum()/len(preds)
print("Full dataset accuracy :", knn_full_acc)

100%|████████████████████████████████████████████████████████████████████████████████| 958/958 [00:29<00:00, 32.26it/s]

Full dataset accuracy : 0.9133611691022965


In [78]:
pca = PCA(n_components=10)
X_train_reduced = pca.fit_transform(X_train)
X_test_reduced = pca.transform(X_test)

knn = KNN(4)
knn.fit(X_train_reduced, y_train)
preds = knn.predict(X_test_reduced, dist_metric='minkowski')

knn10_acc = (y_test.flatten() == preds).sum()/len(preds)
print("Accuracy :", knn10_acc)

100%|███████████████████████████████████████████████████████████████████████████████| 958/958 [00:04<00:00, 208.60it/s]

Accuracy : 0.941544885177453


In [77]:
pca = PCA(n_components=3)
X_train_reduced = pca.fit_transform(X_train)
X_test_reduced = pca.transform(X_test)

max_depth, min_samples_split = 10, 4
knn = KNN(4)
knn.fit(X_train_reduced, y_train)
preds = knn.predict(X_test_reduced, dist_metric='minkowski')

knn3_acc = (y_test.flatten() == preds).sum()/len(preds)
print("Accuracy :", knn3_acc)

100%|███████████████████████████████████████████████████████████████████████████████| 958/958 [00:04<00:00, 212.67it/s]

Accuracy : 0.7630480167014614


In [79]:
pca = PCA(n_components=5)
X_train_reduced = pca.fit_transform(X_train)
X_test_reduced = pca.transform(X_test)

knn = KNN(4)
knn.fit(X_train_reduced, y_train)
preds = knn.predict(X_test_reduced, dist_metric='minkowski')

knn5_acc = (y_test.flatten() == preds).sum()/len(preds)
print("Accuracy :", knn5_acc)

100%|███████████████████████████████████████████████████████████████████████████████| 958/958 [00:04<00:00, 213.15it/s]

Accuracy : 0.9248434237995825


### Comparison of accuracies of two models based on the companents

In [80]:
DataFrame({"Components": ["Full", "3", "5", "10"], "KNN": [knn_full_acc, knn3_acc, knn5_acc, knn10_acc], 
           "Decision Tree": [DTfull_acc, DT3_acc, DT5_acc, DT10_acc] })

,Components,KNN,Decision Tree
0,Full,0.913361,0.896660
1,3,0.763048,0.786013
2,5,0.924843,0.913361
3,10,0.941545,0.911273
